In [ ]:
from clickhouse_driver import Client
import io
import pandas as pd
import numpy as np
from pymongo import MongoClient
from bson.objectid import ObjectId
import datetime
import psycopg2
import requests
import json
import time
from requests_futures.sessions import FuturesSession
from backend_logic import *
session = FuturesSession()

offerid = '5d56807f7e76808f64a3b555'
maxRivalsCount = 12
sqiDiffCoef = 100
maxPos = 10
minCountInSerm = 6

keywords, rivals, startDate, keywordsStr, regionsStr, regions = getRivalsAndData(offerid, maxRivalsCount, sqiDiffCoef, maxPos, minCountInSerm)

In [21]:
from backend_logic import *
offerid = '5d56807f7e76808f64a3b555'
getFilterRivalsObjectByOfferId(offerid)

{'_id': ObjectId('5d56a6e1dc2e09cc9784d846'),
 'keywords': [{'id': 3808050, 'text': 'диван из экокожи'},
  {'id': 3808051, 'text': 'диваны с эко кожи'},
  {'id': 3808052, 'text': 'диван экокожа доставка'},
  {'id': 3808053, 'text': 'диваны из экокожи в москве'},
  {'id': 3808054, 'text': 'диван экокожа цена'},
  {'id': 3808055, 'text': 'купить диван из экокожи'},
  {'id': 3808056, 'text': 'купить диван бежевый экокожа'},
  {'id': 3808057, 'text': 'большой диван экокожа'},
  {'id': 3808058, 'text': 'диван экокожа интернет магазин'},
  {'id': 3808059, 'text': 'купить диван из экокожи цена'},
  {'id': 3808060, 'text': 'диваны экокожа черная'},
  {'id': 3808061, 'text': 'каталог диванов из экокожи'},
  {'id': 3808062, 'text': 'купить диван экокожа маленький'},
  {'id': 3808063, 'text': 'небольшой диван из экокожи'},
  {'id': 3808064, 'text': 'маленькие диваны из экокожи'},
  {'id': 3808065, 'text': 'диван в гостиную экокожа'},
  {'id': 3808066, 'text': 'купить белый диван из экокожи'},
  {

In [ ]:
all_keywords, notRivals = getKeywordsRivals(keywords, rivals, maxPos, startDate, keywordsStr, regionsStr, regions, minCountInSerm)
all_keywords

In [ ]:
minKeywordRivals = 2
keywordsToDelete = getKeywordsToRemove(all_keywords, minKeywordRivals, regions)
keywordsToDelete

In [ ]:
filename = 'Rivals ' + offerid + ' ' + 'manual' + '.xlsx'
info = [{
    'Id КП': offerid,
    'Макс. кол-во конкурентов': maxRivalsCount,
    'Макс. отклонение ИКС': sqiDiffCoef,
    'Макс. позиция для поиска конкурентов': maxPos,
    'Мин. кол-во раз, сколько сайт должен встречаться в выдаче, чтобы считаться конкурентом': minCountInSerm,
    'Мин. кол-во конкурентов для запроса': minKeywordRivals
}]
keywordsToDeleteForExcel = []
for item in keywordsToDelete:
    for key in item['keywordsToDelete']:
        keywordsToDeleteForExcel.append({
            'Регион': item['region'],
            'Ключ для удаления': key
        })
writer = pd.ExcelWriter('./downloadable_files/' + filename)
info_df = pd.DataFrame(info)
info_df.to_excel(writer, index=False, sheet_name='Метаданные', freeze_panes = (1,0))
regions_df = pd.DataFrame(regions).rename(columns={'name': 'Регион'})['Регион'].to_frame()
regions_df.to_excel(writer, index=False, sheet_name='Регионы', freeze_panes = (1,0))
rivals_df = pd.DataFrame(rivals).rename(columns={
    'domain': 'Сайт',
    'isExcluded': 'Не конкурент',
    'countInSerm': 'Встречается в выдаче',
})[['Сайт', 'Не конкурент', 'Встречается в выдаче']]
rivals_df.to_excel(writer, index=False, sheet_name='Конкуренты', freeze_panes = (1,0))
all_keywords_df = pd.DataFrame(keywords).rename(columns={
    'keyword': 'Запрос',
    'regionName': 'Регион',
    'searchEngine': 'ПС',
    'isMobile': 'Мобильная выдача',
    'rivalsCount': 'Кол-во конкурентов'
})[['Запрос', 'Кол-во конкурентов', 'Регион', 'ПС', 'Мобильная выдача']]
all_keywords_df.to_excel(writer, index=False, sheet_name='Запросы', freeze_panes = (1,0))
keywordsToDeleteForExcel_df = pd.DataFrame(keywordsToDeleteForExcel)
keywordsToDeleteForExcel_df.to_excel(writer, index=False, sheet_name='Запросы для удаления', freeze_panes = (1,0))
writer.save()
print(filename)

In [ ]:
keywordsToDeleteForExcel

In [ ]:
keywordsToDeleteForExcel_df = pd.DataFrame(keywordsToDeleteForExcel)
all_keywords_df = pd.DataFrame(keywords).rename(columns={
    'keyword': 'Запрос',
    'regionName': 'Регион',
    'searchEngine': 'ПС',
    'isMobile': 'Мобильная выдача',
    'rivalsCount': 'Кол-во конкурентов'
})[['Запрос', 'Кол-во конкурентов', 'Регион', 'ПС', 'Мобильная выдача']]
keywordsToDeleteForExcel_df